# **LP4_Run2**

In [ ]:
!pip install openai
!pip install python-dotenv
!pip3 install pyomo
!apt install glpk-utils
!pip install glpk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.5/325.5 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 71.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 6.4 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
Suggested packages:
  libiodbc2-dev
The following NEW packages will be installed:
  glpk-utils libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
0 upgraded, 5 newly installed, 0 to remove and 45 not upgraded.
Need to get 625 kB of archives.
After this operation, 2,158 kB of additional disk space will be used.
Ge

In [ ]:

import openai
import os
from IPython.display import Markdown


### **Accessing the GPT4 API**

In [ ]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv('api_file.env'))
my_api_key = os.environ['api_key_env']
openai.api_key = my_api_key


### **Generate Mathematical Model**

In [ ]:
problem = """Consider a farmer who specializes in raising wheat, corn, and sugar beets on her 500 acres of land. During the winter, she wants to decide how much land to devote to each crop.

The farmer knows that at least 200 tons (T) of wheat and 240 T of corn are needed for cattle feed. These amounts can be raised on the farm or bought from a wholesaler.
Any production in excess of the feeding requirement would be sold.

Over the last decade, mean selling prices have been $170 and $150 per ton of wheat and corn, respectively.
The purchase prices are 40% more than this due to the wholesaler’s margin and transportation costs.

Another profitable crop is sugar beet, which she expects to sell at $36/T. However, the European Commission imposes a quota on sugar beet production. Any amount in excess of the quota can be sold only at $10/T. The farmer’s quota for next year is 6000 T.

Based on past experience, the farmer knows that the mean yield on her land is roughly 2.5 T, 3 T, and 20 T per acre for wheat, corn, and sugar beets, respectively. It costs $150 to plant an acre of wheat, $230 to pant an acre of corn and $260 to plant an acre of sugar beets.

The farmer can decide how much of her land she will use to grow each of the three products: wheat, corn and sugar beets. The goal of the farmer is to minimize her total costs (where sales are considered as negative costs)."""

In [ ]:

client = openai.OpenAI(api_key=os.environ['api_key_env'])

response1 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please formulate only the variables for this mathematical optimization problem."},
        {
            "role": "user",
            "content": problem
    }
    ],
    model="gpt-4",
    seed = 2
)


In [ ]:
print(response1.choices[0].message.content)

1. W_a: number of acres of land planted with wheat.
2. C_a: number of acres of land planted with corn.
3. S_a: number of acres of land planted with sugar beets.
4. W_t: tons of wheat produced.
5. C_t: tons of corn produced.
6. S_t: tons of sugar beets produced.
7. W_b: tons of wheat bought from wholesaler.
8. C_b: tons of corn bought from wholesaler.
9. W_s: tons of wheat sold.
10. C_s: tons of corn sold.
11. S_s: tons of sugar beet sold within quota.
12. S_ex: tons of sugar beet sold in excess of quota. 
13. W_p: price per ton of wheat sold.
14. C_p: price per ton of corn sold.
15. S_p: price per ton of sugar beet sold (up to quota).
16. S_ex_p: price per ton of sugar beet sold in excess of quota.
17. W_c: cost to plant an acre of wheat.
18. C_c: cost to plant an acre of corn.
19. S_c: cost to plant an acre of sugar beets.
20. W_pc: purchase price of wheat from wholesaler.
21. C_pc: purchase price of corn from wholesaler.


In [ ]:

response2 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please formulate only the objective function for this mathematical optimization problem."},
        {
            "role": "user",
            "content": problem + response1.choices[0].message.content
        }
    ],
    model="gpt-4",
    seed = 2
)

In [ ]:
print(response2.choices[0].message.content)

The objective function for this optimization problem is:

Minimize Cost = W_a*W_c + C_a*C_c + S_a*S_c + W_b*W_pc + C_b*C_pc - (W_s*W_p + C_s*C_p + S_s*S_p + S_ex*S_ex_p)


In [ ]:
response3 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please formulate only the constraints for this mathematical optimization problem."},
        {
            "role": "user",
            "content": problem + response1.choices[0].message.content +  response2.choices[0].message.content
        }
    ],
    model="gpt-4",
    seed = 2
)

In [ ]:
print(response3.choices[0].message.content)

The constraints for this optimization problem are:

1. Land Constraint: 
   W_a + C_a + S_a ≤ 500 (total areas planted with wheat, corn, and sugar beets should not exceed 500 acres)

2. Crop Production Constraints:
   W_t = 2.5*W_a (wheat tons produced equals 2.5 times the acres planted with wheat)
   C_t = 3*C_a (corn tons produced equals 3 times the acres planted with corn)
   S_t = 20*S_a (sugar beet tons produced equals 20 times the acres planted with sugar beets)

3. Supply-Demand Constraints for Feeds:
   W_b + W_t ≥ 200 (wheat bought and produced should meet the 200T demand)
   C_b + C_t ≥ 240 (corn bought and produced should meet the 240T demand)

4. Selling Constraints:
   W_s ≤ W_t (wheat sold should not exceed wheat produced)
   C_s ≤ C_t (corn sold should not exceed corn produced)
   S_s + S_ex = S_t (sugar beets sold within quota and in excess should equal sugar beet produced)

5. Quota Constraint for Sugar Beets: 
   S_s ≤ 6000 (sugar beets sold within quota should not ex

### **Generate Pyomo Code**

In [ ]:
response4 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please write a python pyomo code for this optimization problem. Use sample data where needed. Indicate where you use sample data."},
        {
            "role": "user",
            "content": response1.choices[0].message.content + response2.choices[0].message.content + response3.choices[0].message.content
        }
    ],
    model="gpt-4",
    seed = 2
)

In [ ]:
Markdown(response4.choices[0].message.content)

Here's the python code using Pyomo to represent the problem:

```python
from pyomo.environ import *

# Sample data (prices, costs, production rates, demand, total land, and quota)
W_p, C_p, S_p, S_ex_p = 170, 150, 36, 10
W_c, C_c, S_c = 150, 230, 260
W_pc, C_pc = 238, 210
W_par, C_par, S_par = 2.5, 3, 20
demand_W, demand_C = 200, 240
total_land = 500
quota_S = 6000

model = ConcreteModel()

# Decision Variables
model.W_a = Var(domain=NonNegativeReals)
model.C_a = Var(domain=NonNegativeReals)
model.S_a = Var(domain=NonNegativeReals)
model.W_b = Var(domain=NonNegativeReals)
model.C_b = Var(domain=NonNegativeReals)
model.W_s = Var(domain=NonNegativeReals)
model.C_s = Var(domain=NonNegativeReals)
model.S_s = Var(domain=NonNegativeReals)
model.S_ex = Var(domain=NonNegativeReals)

# Objective Function
model.Cost = Objective(expr = model.W_a*W_c + model.C_a*C_c + model.S_a*S_c + model.W_b*W_pc + model.C_b*C_pc -
                  (model.W_s*W_p + model.C_s*C_p + model.S_s*S_p + model.S_ex*S_ex_p), sense=minimize)

# Constraints
model.land_constraint = Constraint(expr = model.W_a + model.C_a + model.S_a <= total_land)
model.wheat_produced_constraint = Constraint(expr = model.W_s <= W_par*model.W_a)
model.corn_produced_constraint = Constraint(expr = model.C_s <= C_par*model.C_a)
model.sugar_produced_constraint = Constraint(expr = model.S_s + model.S_ex == S_par*model.S_a)
model.wheat_demand_constraint = Constraint(expr = model.W_b + W_par*model.W_a >= demand_W)
model.corn_demand_constraint = Constraint(expr = model.C_b + C_par*model.C_a >= demand_C)
model.sugar_quota_constraint = Constraint(expr = model.S_s <= quota_S)

# Solve the model
solver = SolverFactory('glpk')
solver.solve(model)

# print results
print("Solution:")
print("Wheat planted (acres): ", model.W_a())
print("Corn planted (acres): ", model.C_a())
print("Sugar beet planted (acres): ", model.S_a())
print("Wheat bought (tons): ", model.W_b())
print("Corn bought (tons): ", model.C_b())
print("Wheat sold (tons): ", model.W_s())
print("Corn sold (tons): ", model.C_s())
print("Sugar beet sold within quota (tons): ", model.S_s())
print("Sugar beet sold in excess of quota (tons): ", model.S_ex())
print("Cost of planting and buying: ", model.Cost())
```
This should return the optimal strategy for planting and purchasing that minimizes cost, subject to supply, demand and quota constraints. The actual values would depend on the specific data input. The solver used is GLPK, a free solver that works well for linear programming problems. You would need to install it and the corresponding python interface if you haven’t already.

### **Run the code Generated by GPT4**

In [ ]:
from pyomo.environ import *

# Sample data (prices, costs, production rates, demand, total land, and quota)
W_p, C_p, S_p, S_ex_p = 170, 150, 36, 10
W_c, C_c, S_c = 150, 230, 260
W_pc, C_pc = 238, 210
W_par, C_par, S_par = 2.5, 3, 20
demand_W, demand_C = 200, 240
total_land = 500
quota_S = 6000

model = ConcreteModel()

# Decision Variables
model.W_a = Var(domain=NonNegativeReals)
model.C_a = Var(domain=NonNegativeReals)
model.S_a = Var(domain=NonNegativeReals)
model.W_b = Var(domain=NonNegativeReals)
model.C_b = Var(domain=NonNegativeReals)
model.W_s = Var(domain=NonNegativeReals)
model.C_s = Var(domain=NonNegativeReals)
model.S_s = Var(domain=NonNegativeReals)
model.S_ex = Var(domain=NonNegativeReals)

# Objective Function
model.Cost = Objective(expr = model.W_a*W_c + model.C_a*C_c + model.S_a*S_c + model.W_b*W_pc + model.C_b*C_pc -
                  (model.W_s*W_p + model.C_s*C_p + model.S_s*S_p + model.S_ex*S_ex_p), sense=minimize)

# Constraints
model.land_constraint = Constraint(expr = model.W_a + model.C_a + model.S_a <= total_land)
model.wheat_produced_constraint = Constraint(expr = model.W_s <= W_par*model.W_a)
model.corn_produced_constraint = Constraint(expr = model.C_s <= C_par*model.C_a)
model.sugar_produced_constraint = Constraint(expr = model.S_s + model.S_ex == S_par*model.S_a)
model.wheat_demand_constraint = Constraint(expr = model.W_b + W_par*model.W_a >= demand_W)
model.corn_demand_constraint = Constraint(expr = model.C_b + C_par*model.C_a >= demand_C)
model.sugar_quota_constraint = Constraint(expr = model.S_s <= quota_S)

# Solve the model
solver = SolverFactory('glpk')
solver.solve(model)

# print results
print("Solution:")
print("Wheat planted (acres): ", model.W_a())
print("Corn planted (acres): ", model.C_a())
print("Sugar beet planted (acres): ", model.S_a())
print("Wheat bought (tons): ", model.W_b())
print("Corn bought (tons): ", model.C_b())
print("Wheat sold (tons): ", model.W_s())
print("Corn sold (tons): ", model.C_s())
print("Sugar beet sold within quota (tons): ", model.S_s())
print("Sugar beet sold in excess of quota (tons): ", model.S_ex())
print("Cost of planting and buying: ", model.Cost())

Solution:
Wheat planted (acres):  120.0
Corn planted (acres):  80.0
Sugar beet planted (acres):  300.0
Wheat bought (tons):  0.0
Corn bought (tons):  0.0
Wheat sold (tons):  300.0
Corn sold (tons):  240.0
Sugar beet sold within quota (tons):  6000.0
Sugar beet sold in excess of quota (tons):  0.0
Cost of planting and buying:  -188600.0


### **Edit and Run the code for the mathematical model produced by GPT4 (Circumstantial)**